In [1]:
import pandas as pd
df01 = pd.read_csv('C:/Users/user/Desktop/workSpace/분석/표준분석모델/인천광역시_동구_전기차/work2/인천광역시_동구_최종_데이터전처리.csv',
                   encoding = 'cp949' ,engine='python')
df01.set_index('gid', inplace=True)
전체칼럼 = df01.columns.tolist()
print ("여기 칼럼명을 확인하신 후 점수계산에 필요한 칼럼명만 복사 하세요.\n", 전체칼럼)

여기 칼럼명을 확인하신 후 점수계산에 필요한 칼럼명만 복사 하세요.
 ['주차장', '건물1', '건물2', '건물3', '경제인구', '관공서', '전기차', '충전소', '경도', '위도', '거리_충전소', '거리_전기차']


In [2]:
# 가중치와 합계에 필요한 칼럼 추출
점수칼럼 =  ['주차장', '건물1', '건물2', '건물3', '경제인구', '관공서', '전기차', '충전소', '거리_충전소', '거리_전기차']
print("입력한 점수컬럼명은 다음과 같음\n", 점수칼럼)

가중치 = [1] * len(점수칼럼)

입력한 점수컬럼명은 다음과 같음
 ['주차장', '건물1', '건물2', '건물3', '경제인구', '관공서', '전기차', '충전소', '거리_충전소', '거리_전기차']


In [3]:
# 가중치 배정
가중치 = [1, 1, 2, 3, 1, 3, 2, -3, -3, 2]


# 수정 불필요
if (len(가중치) != len(점수칼럼)):
    print("입력한 가중치 갯수가 점수칼럼 갯수와 다르므로 확인후 다시 입력 바랍니다")
else:
    print("다음 줄을 실행 하세요")

다음 줄을 실행 하세요


In [8]:
df = pd.read_csv('C:/Users/user/Desktop/workSpace/분석/표준분석모델/인천광역시_동구_전기차/work2/인천광역시_동구_최종_데이터전처리.csv',
                 encoding = 'cp949' ,engine='python')
df.set_index('gid', inplace=True)

################## QGIS에서 작업한 전처리 결과 CSV 읽기 ####################################################
pre_process_set = pd.read_csv('C:/Users/user/Desktop/workSpace/분석/표준분석모델/인천광역시_동구_전기차/work2/인천광역시_동구_최종_데이터전처리.csv',
                              encoding = 'cp949' ,engine='python')
pre_process_set = pre_process_set.sort_values(by='gid')

# Null 이 있을 수 있는 칼럼은 Null을 0로 변경
pre_process_set.fillna(0, inplace=True)
##################  QGIS에서 작업한 전처리 결과 CSV 읽기 끝###################################################

# 점수칼럼 정규화
norm = pre_process_set.copy()
norm.fillna(0, inplace=True)
norm.set_index('gid', inplace=True)    # gid를 인덱스 설정

norm[점수칼럼] = (norm.loc[:,점수칼럼] - norm.loc[:,점수칼럼].mean())/norm.loc[:,점수칼럼].std()
norm.reset_index(inplace=True)    # 설정한 인덱스를 해제
################## 정규화 끝################################################################################


df = norm.copy()

for i in range(0, len(점수칼럼)):
    name = 점수칼럼[i]
    df[name] = df[name] * 가중치[i] 

df.set_index(['gid','경도','위도'], inplace=True)

df['sum'] = df.sum(1)
df.reset_index(inplace=True)

# 등수 계산 
df['순위'] = df['sum'].rank(method='dense', ascending=False).astype(int)

final = df.sort_values(by='순위')
final.reset_index(drop=True, inplace=True)

# 상위 20개만 학인
display("결과", final.head(20))

# 최종 결과 CSV 파일쓰기
final.to_csv('C:/Users/user/Desktop/workSpace/분석/표준분석모델/인천광역시_동구_전기차/work2/Z_score결과_총점.csv', index=False, sep=',', encoding='cp949')

'결과'

,gid,경도,위도,주차장,건물1,건물2,건물3,경제인구,관공서,전기차,충전소,거리_충전소,거리_전기차,sum,순위
0,다사239419,126.639920,37.473647,-0.487299,0.401548,1.800620,66.410713,2.378700,-0.660159,-0.270911,-16.667223,-5.177078,2.655726,50.384638,1
1,다사240419,126.641051,37.473656,1.334776,0.401548,1.800620,33.108548,1.573808,10.257849,-0.270911,0.417177,-5.177078,2.655726,46.102064,2
2,다사249426,126.651157,37.480038,-0.487299,-0.539410,6.787844,-0.193617,5.299823,10.257849,20.909383,0.417177,-5.177078,2.655726,39.930399,3
3,다사236424,126.636476,37.478129,-0.487299,-0.539410,9.281456,-0.193617,6.810125,10.257849,8.201207,0.417177,1.736411,2.655726,38.139626,4
4,다사242419,126.643312,37.473672,-0.031780,-0.001719,-0.692992,-0.193617,-0.388680,10.257849,42.089676,-16.667223,-5.177078,2.655726,31.850161,5
5,다사223439,126.621616,37.491540,-0.031780,-0.136142,-0.692992,33.108548,-0.442943,-0.660159,-0.270911,0.417177,1.736411,-1.504428,31.522782,6
6,다사243428,126.644351,37.481792,-0.031780,-0.270565,11.775069,-0.193617,4.142226,10.257849,-0.270911,0.417177,1.736411,2.655726,30.217585,7
7,다사223431,126.621700,37.484330,-0.487299,0.267126,1.800620,-0.193617,1.275365,10.257849,12.437265,0.417177,1.736411,2.655726,30.166623,8
8,다사235423,126.635355,37.477220,0.423738,-0.136142,6.787844,-0.193617,3.662909,10.257849,-0.270911,0.417177,1.736411,2.655726,25.340985,9
9,다사248427,126.650016,37.480931,-0.487299,-0.270565,9.281456,-0.193617,2.677143,-0.660159,16.673324,0.417177,-5.177078,2.655726,24.916110,10


In [9]:
df = pd.read_csv('C:/Users/user/Desktop/workSpace/분석/표준분석모델/인천광역시_동구_전기차/work2/인천광역시_동구_최종_데이터전처리.csv',
                 encoding = 'cp949' ,engine='python')
df.set_index('gid', inplace=True)

################## QGIS에서 작업한 전처리 결과 CSV 읽기 ####################################################
pre_process_set = pd.read_csv('C:/Users/user/Desktop/workSpace/분석/표준분석모델/인천광역시_동구_전기차/work2/인천광역시_동구_최종_데이터전처리.csv',
                              encoding = 'cp949' ,engine='python')
pre_process_set = pre_process_set.sort_values(by='gid')

# Null 이 있을 수 있는 칼럼은 Null을 0로 변경
pre_process_set.fillna(0, inplace=True)
##################  QGIS에서 작업한 전처리 결과 CSV 읽기 끝###################################################

# 점수칼럼 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# 점수칼럼 정규화
norm = pre_process_set.copy()
norm.fillna(0, inplace=True)
norm.set_index('gid', inplace=True)

norm[점수칼럼] = scaler.fit_transform(norm.loc[:,점수칼럼])
norm.reset_index(inplace=True)
#################################################


df = norm.copy()

for i in range(0, len(점수칼럼)):
    name = 점수칼럼[i]
    df[name] = df[name] * 가중치[i] 

df.set_index(['gid','경도','위도'], inplace=True)

df['sum'] = df.sum(1)
df.reset_index(inplace=True)

# 등수 계산 
df['순위'] = df['sum'].rank(method='dense', ascending=False).astype(int)

final = df.sort_values(by='순위')
final.reset_index(drop=True, inplace=True)

# 상위 20개만 학인
display("결과", final.head(20))

# 최종 결과 CSV 파일쓰기
final.to_csv('C:/Users/user/Desktop/workSpace/분석/표준분석모델/인천광역시_동구_전기차/work2/MinMax결과_총점.csv', index=False, sep=',', encoding='cp949')

'결과'

,gid,경도,위도,주차장,건물1,건물2,건물3,경제인구,관공서,전기차,충전소,거리_충전소,거리_전기차,sum,순위
0,다사236424,126.636476,37.478129,0.00,0.000000,1.333333,0.0,0.853191,1.0,0.4,-0.0,-0.0,2.0,5.586525,1
1,다사243428,126.644351,37.481792,0.04,0.028571,1.666667,0.0,0.539362,1.0,0.0,-0.0,-0.0,2.0,5.274600,2
2,다사235423,126.635355,37.477220,0.08,0.042857,1.000000,0.0,0.482979,1.0,0.0,-0.0,-0.0,2.0,4.605836,3
3,다사237424,126.637606,37.478137,0.00,0.000000,1.333333,0.0,1.000000,0.0,0.0,-0.0,-0.0,2.0,4.333333,4
4,다사241428,126.642089,37.481775,0.00,0.000000,2.000000,0.0,0.279787,0.0,0.0,-0.0,-0.0,2.0,4.279787,5
5,다사223431,126.621700,37.484330,0.00,0.085714,0.333333,0.0,0.202128,1.0,0.6,-0.0,-0.0,2.0,4.221175,6
6,다사247426,126.648895,37.480022,0.00,0.000000,1.333333,0.0,0.688298,0.0,0.0,-0.0,-0.0,2.0,4.021631,7
7,다사247418,126.648977,37.472812,0.00,0.000000,0.000000,0.0,0.000000,2.0,0.0,-0.0,-0.0,2.0,4.000000,8
8,다사248426,126.650026,37.480030,0.00,0.000000,1.333333,0.0,0.665957,0.0,0.0,-0.0,-0.0,2.0,3.999291,9
9,다사242428,126.643220,37.481784,0.04,0.000000,1.666667,0.0,0.207447,0.0,0.0,-0.0,-0.0,2.0,3.914113,10


In [9]:
# 등수 계산 
#df1000['순위'] = df1000['sum'].rank(method='dense', ascending=False).astype(int)

df1000['순위'] = df1000['sum'].rank(method='min', ascending = False).astype(int)

# final = df1000.sort_values(by='순위')
# final.reset_index(drop=True, inplace=True)

In [12]:
display("결과", df1000.head(20))

'결과'

,gid,경도,위도,주차장,건물1,건물2,건물3,경제인구,관공서,교통량,변압기,전기차,충전소,sum,순위
0,다사62bb66ba,127.078871,37.698435,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
1,다사63aa66aa,127.081732,37.693938,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
2,다사63aa66ab,127.081719,37.696192,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
3,다사63aa66ba,127.081706,37.698445,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
4,다사63aa66bb,127.081694,37.700698,-0.074092,-0.332167,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.338996,3778
5,다사63ab65ba,127.084593,37.689442,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
6,다사63ab65bb,127.084580,37.691695,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
7,다사63ab66aa,127.084568,37.693948,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
8,다사63ab66ab,127.084555,37.696202,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
9,다사63ab66ba,127.084542,37.698455,-0.074092,-0.466883,-0.38429,-0.176376,-0.237898,-0.394969,-0.594403,-0.06831,-0.271336,0.194845,-2.473713,4085
